In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import os

from sklearn.model_selection import StratifiedKFold
from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.nn.models import Node2Vec
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GINConv
from torch.optim import Adam
from torch.nn import NLLLoss
from collections import defaultdict
from dateutil import parser

import ast
import preprocessing
import utils

torch.manual_seed(0)
np.random.seed(0)

profiles = pd.read_csv("../data/profiles.csv", usecols=["profile_username", "profile_followed_by", "profile_follow", 
                                                        "medias_nb", "comments_nb", "comments_commenters_nb", 
                                                        "comments_self_nb", "category_1"])
comments = pd.read_csv("../data/comments.csv")

In [2]:
profiles["is_tracked"] = 1

In [3]:
final_df = comments[["media_author", "commenter"]]\
                .groupby("commenter", as_index=False)\
                .agg("count")\
                .sort_values("commenter")
final_df.columns = ["commenter", "comments_nb"]
final_df = final_df[(final_df.comments_nb >= 125) | final_df.commenter.isin(profiles.profile_username)]

In [4]:
print("The number of commenters: {}".format(len(final_df)))
comments = comments[comments.commenter.isin(final_df.commenter.values)]
print("The current number of interactions: {}".format(len(comments)))

The number of commenters: 3583
The current number of interactions: 882651


In [5]:
tmp = comments[["category_1", "commenter"]]\
            .groupby("commenter", as_index=False)\
            .agg({"category_1": lambda col: col.nunique()})\
            .sort_values("commenter")

to_categories_nb = {commenter: categories_nb for commenter, categories_nb in tmp.values}
final_df["categories_nb"] = final_df.commenter.apply(lambda commenter: to_categories_nb[commenter])

del tmp, to_categories_nb

In [6]:
tmp = comments[["commenter", "comment_tags"]]
tmp["comment_tags"] = tmp.comment_tags.apply(lambda x: len(ast.literal_eval(x)))
tmp = tmp.groupby("commenter", as_index=False)\
            .agg("mean")\
            .sort_values("commenter")

to_avg_tags_nb = {commenter: avg_tags_nb for commenter, avg_tags_nb in tmp.values}
final_df["avg_tags_nb"] = final_df.commenter.apply(lambda commenter: to_avg_tags_nb[commenter])

del tmp, to_avg_tags_nb

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
tmp = comments[["commenter", "comment_mentioned_usernames"]]
tmp["comment_mentioned_usernames"] = tmp.comment_mentioned_usernames.apply(lambda x: len(ast.literal_eval(x)))
tmp = tmp.groupby("commenter", as_index=False)\
            .agg("mean")\
            .sort_values("commenter")

to_avg_cited_users = {commenter: avg_cited_users for commenter, avg_cited_users in tmp.values}
final_df["avg_cited_users"] = final_df.commenter.apply(lambda commenter: to_avg_cited_users[commenter])

del tmp, to_avg_cited_users

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
tmp = comments[["commenter", "comment_text"]]
tmp["comment_text"] = tmp.comment_text.apply(lambda x: len(x))
tmp = tmp.groupby("commenter", as_index=False)\
            .agg("mean")\
            .sort_values("commenter")

to_avg_msg_len = {commenter: avg_msg_len for commenter, avg_msg_len in tmp.values}
final_df["avg_msg_len"] = final_df.commenter.apply(lambda commenter: to_avg_msg_len[commenter])

del tmp, to_avg_msg_len

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
def update_dict(to_weekday, commenter, day, freq):
    if to_weekday[commenter][1] < freq:
        to_weekday[commenter] = (day, freq)

In [10]:
tmp = comments[["commenter", "comment_created_time_str"]]
tmp["day"] = tmp.comment_created_time_str.apply(lambda item: parser.parse(item).weekday())
tmp = tmp.groupby(["commenter", "day"], as_index=False).agg("count")

to_weekday = {}
for commenter, day, freq in tmp.values:
    if not commenter in to_weekday:
        to_weekday[commenter] = (0, 0)
    
    update_dict(to_weekday, commenter, day, freq)
    
final_df["weekday"] = final_df.commenter.apply(lambda commenter: to_weekday[commenter][0])

del tmp, to_weekday

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
tmp = comments[["commenter", "comment_created_time_str"]]
tmp["hour"] = tmp.comment_created_time_str.apply(lambda item: parser.parse(item).hour)
tmp = tmp.groupby(["commenter", "hour"], as_index=False).agg("count")

to_hour = {}
for commenter, hour, freq in tmp.values:
    if not commenter in to_hour:
        to_hour[commenter] = (0, 0)
    
    update_dict(to_hour, commenter, hour, freq)
    
final_df["hour"] = final_df.commenter.apply(lambda commenter: to_hour[commenter][0])

del tmp, to_hour

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
final_df["is_tracked"] = 0

In [13]:
final_df.head(5)

,commenter,comments_nb,categories_nb,avg_tags_nb,avg_cited_users,avg_msg_len,weekday,hour,is_tracked
687,0591_aleinad,259,1,0.000000,0.814672,196.810811,0,16,0
860,06nazi,126,1,0.000000,0.015873,5.238095,6,18,0
1747,102.7245,218,2,0.000000,0.059633,36.885321,6,12,0
2210,11luisella,397,1,0.020151,0.020151,46.188917,5,12,0
2445,12nycole,222,2,1.648649,1.500000,203.639640,5,12,0


In [14]:
cols = set(final_df.columns).union(profiles.columns)
for col in cols:
    if not col in final_df.columns and col != "profile_username":
        final_df[col] = 0
    if not col in profiles.columns and col != "commenter":
        profiles[col] = 0

n_features = len(set(final_df.columns).union(profiles.columns))

In [15]:
final_df.rename(columns={'commenter':'profile_username'}, inplace=True)
final_df = final_df.reindex(sorted(final_df.columns), axis=1)
profiles = profiles.reindex(sorted(profiles.columns), axis=1)

cols = profiles.columns

In [16]:
final_df.head(1)

,avg_cited_users,avg_msg_len,avg_tags_nb,categories_nb,category_1,comments_commenters_nb,comments_nb,comments_self_nb,hour,is_tracked,medias_nb,profile_follow,profile_followed_by,profile_username,weekday
687,0.814672,196.810811,0.0,1,0,0,259,0,16,0,0,0,0,0591_aleinad,0


In [17]:
user_dict = {values[-2]: values for values in profiles.values}

In [18]:
remaining_users = list(set(profiles.profile_username) - set(final_df.profile_username))

profiles = profiles[profiles.profile_username.isin(remaining_users)].values
final_df = final_df.values

In [19]:
def merge_row(row1, row2):
    for i in range(row1.size):
        if type(row2[i]) is str:
            row1[i] = row2[i]
        elif type(row1[i]) is str:
            continue
        elif row1[i] == 0:
            row1[i] += row2[i]

In [20]:
for row in final_df:
    name = row[-2]
    if name in user_dict:
        merge_row(row, user_dict[name])

list(final_df).append(profiles)

In [21]:
final_df = pd.DataFrame(final_df, columns=cols)

In [22]:
final_df.head(1)

,avg_cited_users,avg_msg_len,avg_tags_nb,categories_nb,category_1,comments_commenters_nb,comments_nb,comments_self_nb,hour,is_tracked,medias_nb,profile_follow,profile_followed_by,profile_username,weekday
0,0.814672,196.811,0,1,0,0,259,0,16,0,0,0,0,0591_aleinad,0


In [23]:
final_df.to_csv("../data/new_profiles.csv")

In [26]:
users = final_df.profile_username.values
comments = pd.read_csv("../data/comments.csv", usecols=["media_short_code", "media_author", "commenter"])
comments = comments[(comments.commenter.isin(users)) | (comments.media_author.isin(users))]
comments.to_csv("../data/new_comments.csv")

In [ ]:
1a

In [ ]:
results = profiles.category_1.value_counts()
print(results)

figure = go.Figure(
    data=[go.Pie(labels=results.index.values, values=results.values)],
    layout_title_text="Percentage of Each Category"
)

figure.show()

In [ ]:
known_users = profiles.profile_username.unique().tolist()
followers = comments.commenter.unique().tolist()
all_users = set(known_users + followers)
prev_number_of_users = len(all_users)
print("There are originally {} users".format(prev_number_of_users))

In [ ]:
all_posts = len(comments.media_short_code.unique())
all_interactions = len(comments)
print("There are {} distinct posts and {} interactions. An average of {} interactions per post"\
          .format(all_posts, all_interactions, np.round(all_interactions/all_posts, 2)))

In [ ]:
profiles = preprocessing.categorical_to_numerical(profiles, col="category_1")
comments = comments.drop_duplicates()
comments = preprocessing.filter_by_relevance(comments, profiles, minimum_freq=50)

In [ ]:
known_users = profiles.profile_username.unique().tolist()
followers = comments.commenter.unique().tolist()
all_users = set(known_users + followers)
cur_number_of_users = len(all_users)
print("The new graph drawn from relevance threshold {} has {} users and {} interactions"\
          .format(35, cur_number_of_users, len(comments)))

In [ ]:
print("The number of users was reduced by ~ {}%"\
          .format(np.round((1-cur_number_of_users/prev_number_of_users)*100, 2)))

In [ ]:
names = profiles.profile_username.values
data = profiles[["profile_followed_by", "profile_follow", "medias_nb", 
                "comments_nb", "comments_commenters_nb", "comments_self_nb"]].values
name_to_record = {name: record for name, record in zip(names, data)}

input_dim, output_dim = data.shape[1], len(profiles.category_1.unique()) + 1
user_to_label = {user: category for user, category in profiles[["profile_username", "category_1"]].values}

In [ ]:
def update_histories(models_histories, new_histories):
    for model, history in new_histories.items():
        if not models_histories[model]:
            models_histories[model] = np.array(new_histories[model])
        else:
            models_histories[model] += np.array(new_histories[model])
    
    return models_histories
    

def calculate_statistics(models_metrics):
    return {model: {metric: (np.mean(values), np.std(values)) for metric, values in metrics.items()} 
                for model, metrics in models_metrics.items()}

In [ ]:
K = 5
skf = StratifiedKFold(n_splits=K)

n_hidden_units = 64
print("Fez of preprocessamento")
models_metrics = defaultdict(dict)
models_histories = defaultdict(list)
for train_idx, test_idx in skf.split(profiles.profile_username.values, profiles.category_1.values):
    train_authors, test_authors = utils.get_authors(profiles, all_users, train_idx, test_idx)

    print("Pegou autores")

    username_to_index = utils.get_users_indices(train_authors)
    print("Pegou indices")
    train_interactions = utils.get_interactions(comments[(comments.media_author.isin(train_authors)) 
                                                    & (comments.commenter.isin(train_authors))], username_to_index)
    print("Pegou interações")
    x_train, y_train = utils.get_x(train_authors, name_to_record, input_dim=input_dim), utils.get_y(user_to_label, train_authors)
    print("Pegou x e y")
    assert len(x_train)==len(y_train), "Train Input and Output tensor do not have the same dimensions"


    edge_index = utils.get_edge_index(train_interactions)
    print("Pegou os indices de arestas")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = Data(x=x_train, y=y_train, edge_index=edge_index).to(device)
    print("Criou Data")

    models = utils.get_models(data.num_nodes, input_dim, output_dim, n_hidden_units, device=device, lr=0.005)
    print("Criou modelos")

    histories = utils.train(data, models, epochs=1)
    models_histories = update_histories(models_histories, histories)

    username_to_index = utils.get_users_indices(test_authors)
    test_interactions = utils.get_interactions(comments[(comments.media_author.isin(test_authors)) 
                                                    & (comments.commenter.isin(test_authors))], username_to_index)
    x_test, y_test = utils.get_x(test_authors, name_to_record, input_dim=input_dim), utils.get_y(user_to_label, test_authors)
    assert len(x_test)==len(y_test), "Test Input and Output tensor do not have the same dimensions"

    edge_index = utils.get_edge_index(test_interactions)
    data = Data(x=x_test, y=y_test, edge_index=edge_index).to(device)
    current_metrics = utils.test(data, models)
    utils.update_metrics_dict(models_metrics, current_metrics)
    
models_histories = {model: list(history/K) for model, history in models_histories.items()} # Get mean traces

In [ ]:
models_metrics = calculate_statistics(models_metrics)

In [ ]:
import matplotlib.pyplot as plt
import json

models_histories = json.load(open("../pipeline/results/one_layer_models/models_histories_1_256.json"))

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = (15, 5)

fig, ax = plt.subplots(1, 3, sharey=True)
models = ["GCN", "GAT", "GraphSAGE"]
for i, (model, history) in enumerate(models_histories.items()):
    ax[i].plot(range(len(history)), history)
    ax[i].set_title(models[i])

plt.suptitle("History of Training Error (200 Epochs)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import json

models_histories = json.load(open("../pipeline/results/one_layer_models/models_metrics_1_64.json"))
models_histories